# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902058


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:01<00:25,  1.08it/s]

Rendering models:  10%|█         | 3/30 [00:05<00:48,  1.81s/it]

Rendering models:  13%|█▎        | 4/30 [00:05<00:34,  1.31s/it]

Rendering models:  17%|█▋        | 5/30 [00:08<00:39,  1.57s/it]

...

Rendering models:  83%|████████▎ | 25/30 [00:13<00:01,  3.18it/s]

Rendering models:  93%|█████████▎| 28/30 [00:13<00:00,  3.84it/s]

Rendering models:  97%|█████████▋| 29/30 [00:14<00:00,  3.60it/s]

Rendering models: 100%|██████████| 30/30 [00:14<00:00,  4.29it/s]

kayleebug2017                         0.000531
equidad1                              0.104159
Lukes0404                             0.000655
ElisabethB                            0.000126
not-logged-in-8c0c44faf754d344b18a    0.000181
Thedunkmasta                          0.003735
not-logged-in-0997cfcba8da23453381    0.143382
not-logged-in-20487e5808245554c355    0.000144
not-logged-in-ab1612bc18cd7aab85db    0.000114
not-logged-in-2ef7f191c8c973ce093e    0.000145
not-logged-in-499e6717ee7f2a15ae8b    0.000242
not-logged-in-5f30b2bb9b7f9d49da41    0.017702
not-logged-in-61ad0e6cee4d7cae36cc    0.000188
clairedeu                             0.000244
Planetme                              0.000453
ShirleyL                              0.000136
helios1235                            0.000098
Lavadude                              0.064684
not-logged-in-4a99265f22da29adddc5    0.004953
Denny2507                             0.003839
gmann2023                             0.000867
Knight6409   

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())